In [31]:
import pickle
import random

matches_df = pickle.load(open("database/matches_df.pck", "rb"))

In [32]:
winner_list = []
market_val_diff = []
goals_diff = []
for _, row in matches_df.iterrows():
    if row["Team Home ID"] == row["Winner Team ID"]:
        winner_list.append("Home")
    elif row["Team Away ID"] == row["Winner Team ID"]:
        winner_list.append("Away")
    elif row["Winner Team ID"] == 0:
        winner_list.append("Draw")
    market_val_diff.append(row["Market Value Home"] - row["Market Value Away"])
    goals_diff.append(row["Goals Home"] - row["Goals Away"])
matches_df["Result"] = winner_list
matches_df["Market Value Difference"] = market_val_diff
matches_df["Goals Difference"] = goals_diff

In [33]:
y_pred = []
for _, row in matches_df.iterrows():
    if row["Market Value Difference"] >= 0:
        y_pred.append([2, 1, 1])
    else:
        y_pred.append([1, 2, -1])

In [34]:
points = 0
point_list = []
for index, row in matches_df.iterrows():
    pred_home_goals, pred_away_goals, pred_goal_diff = y_pred[index]
    if pred_home_goals == row["Goals Home"] and pred_away_goals == row["Goals Away"]:
        points += 5
        point_list.append(5)
    elif pred_goal_diff == row["Goals Difference"]:
        points += 3
        point_list.append(3)
    elif (pred_goal_diff > 0 and row["Goals Difference"] > 0) or (pred_goal_diff < 0 and row["Goals Difference"] < 0):
        points += 1
        point_list.append(1)
    else:
        point_list.append(0)
points / 10

324.9

In [35]:
import pandas as pd
df = pd.concat([matches_df[["Market Value Difference","Goals Home", "Goals Away", "Goals Difference"]], pd.DataFrame(y_pred, columns=["Pred Goals Home", "Pred Goals Away", "Pred Goals Difference"])], axis=1)
df["Points"] = point_list

In [36]:
df["Points"].value_counts()/10

Points
0    151.5
1     94.2
3     35.4
5     24.9
Name: count, dtype: float64

In [37]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(matches_df)

In [54]:
last = 900
range_dicts = dict()
for i in range(700, -900, -50):
    filtered_df = train_df[(train_df["Market Value Difference"] > i) & (train_df["Market Value Difference"] < last)]
    if len(filtered_df) == 0:
        pass
    else:
        range_dicts[str(last) + "," + str(i)] = filtered_df["Result"].value_counts()/len(filtered_df)
        last = i

In [86]:
import numpy

range_dict_keys = range_dicts.keys()
y_pred = []
val_i = 0
row_i = 0
for _, row in test_df.iterrows():
    for key in range_dict_keys:
        interval = [int(x) for x in key.split(",")]
        if interval[0] > row["Market Value Difference"] > interval[1]:
            pred_res = numpy.random.choice(list(range_dicts[key].index), p=list(range_dicts[key].values))
            y_pred.append(pred_res)
            val_i += 1
            pass
    row_i += 1
    if val_i != row_i:
        pred_res = numpy.random.choice(["Home", "Away", "Draw"])
        y_pred.append(pred_res)
        val_i += 1

ValueError: a must be 1-dimensional

In [56]:
from sklearn.metrics import *

accuracy_score(test_df["Result"], y_pred)

0.40522875816993464

In [57]:
confusion_matrix(test_df["Result"], y_pred)

array([[ 82,  53,  72],
       [ 63,  47,  85],
       [ 88,  94, 181]])

In [66]:
last = 900
range_dicts_exact = dict()
for i in range(700, -900, -50):
    filtered_df = train_df[(train_df["Market Value Difference"] > i) & (train_df["Market Value Difference"] < last)]
    if len(filtered_df) == 0:
        pass
    else:
        range_dicts_exact[str(last) + "," + str(i)] = filtered_df.groupby(by=["Goals Home", "Goals Away"]).size()/len(filtered_df)
        last = i

In [100]:
import numpy

range_dict_exact_keys = range_dicts.keys()
y_pred = []
val_i = 0
row_i = 0
for _, row in test_df.iterrows():
    for key in range_dict_exact_keys:
        interval = [int(x) for x in key.split(",")]
        if interval[0] > row["Market Value Difference"] > interval[1]:
            pred_res = numpy.random.choice([str(x) for x in list(range_dicts_exact[key].index)], p=list(range_dicts[key].values))
            y_pred.append(pred_res)
            val_i += 1
            pass
    row_i += 1
    if val_i != row_i:
        pred_res = numpy.random.choice(["(2, 1)", "(1, 1)", "(1, 2)"])
        y_pred.append(pred_res)
        val_i += 1

In [101]:
y_pred = [[int(y[0]), int(y[1]), int(y[0]) - int(y[1])] for y in [str(x).replace("(", "").replace(")", "").split(",") for x in y_pred]]

In [102]:
points = 0
point_list = []
for index, (_, row) in enumerate(test_df.iterrows()):
    pred_home_goals, pred_away_goals, pred_goal_diff = y_pred[index]
    if pred_home_goals == row["Goals Home"] and pred_away_goals == row["Goals Away"]:
        points += 5
        point_list.append(5)
    elif pred_goal_diff == row["Goals Difference"]:
        points += 3
        point_list.append(3)
    elif (pred_goal_diff > 0 and row["Goals Difference"] > 0) or (pred_goal_diff < 0 and row["Goals Difference"] < 0):
        points += 1
        point_list.append(1)
    else:
        point_list.append(0)
points * 306 / len(test_df)

263.2